# Higher-level group analysis

This notebook demonstrates the higher (second) level group analysis steps for the synthetic dataset.

In __Part 1__ it uses the interactive python interface to perform the analysis.  
In __Part 2__ it demonstrates the equivalent command line steps.

Plotting of the group results is demonstrated.

# Part 1
## 1. First-level (single-subject) contrasts

The first step of this process is actually to construct contrasts of interest from the raw fitted betas of each subject.  
At the same time we also take the opportunity to sum any metabolites which typically show substantial (negative) correlation.

Tools for fMRS stats are contained in the `fsl_mrs.utils.fmrs_tools` package of FSL-MRS.


### 1.1 Assemble the locations of the first-level fits.
We want all the stim results and then all the ctrl results, each sorted by subject number. This simplifies making the higher-level design matrices later.

In [1]:
from pathlib import Path
# Create a list of the results directories
firstlevel_results_dir = Path('first_level_results')

subject_results_stim = []
subject_results_ctrl = []
for file in firstlevel_results_dir.rglob('free_parameters.csv'):
    if file.parent.name == 'stim':
        subject_results_stim.append(str(file.parent))
    else:
        subject_results_ctrl.append(str(file.parent))

subject_results = sorted(subject_results_stim) + sorted(subject_results_ctrl)
print(subject_results)

['first_level_results/sub0/stim', 'first_level_results/sub1/stim', 'first_level_results/sub2/stim', 'first_level_results/sub3/stim', 'first_level_results/sub4/stim', 'first_level_results/sub5/stim', 'first_level_results/sub6/stim', 'first_level_results/sub7/stim', 'first_level_results/sub8/stim', 'first_level_results/sub9/stim', 'first_level_results/sub0/ctrl', 'first_level_results/sub1/ctrl', 'first_level_results/sub2/ctrl', 'first_level_results/sub3/ctrl', 'first_level_results/sub4/ctrl', 'first_level_results/sub5/ctrl', 'first_level_results/sub6/ctrl', 'first_level_results/sub7/ctrl', 'first_level_results/sub8/ctrl', 'first_level_results/sub9/ctrl']


### 1.2 Describe the new contrasts we are going to make

This includes summing the betas of highly (negatively) correlated peaks, e.g. NAA & NAAG, Cr & PCr, and averaging the two activation block betas (0 & 1).

There is a dataclass to help with the description of the contrasts.

In [3]:
import fsl_mrs.utils.fmrs_tools as fmrs

# Form mean activation contrasts take mean of two betas (i.e. 0.5 * beta0 + 0.5 * beta1)
contrasts = [
    fmrs.Contrast(
        'mean_activation',
        ['beta0', 'beta1'],
        [0.5, 0.5])]

metabo_comb = [['PCh','GPC'],
               ['Cr','PCr'],
               ['NAA', 'NAAG'],
               ['Glu', 'Gln']]

### 1.3 Use the FSL-MRS tools to linearly combine values and propagate variances.

We save the output betas (which include the original ones as well) and the variances on each beta.

In [4]:
import numpy as np

modified_betas = []
modified_var = []

for res_path in subject_results:
    _, _, df, new_params = fmrs.create_contrasts(res_path, contrasts=contrasts, metabolites_to_combine=metabo_comb)

    modified_betas.append(df['mean'].to_numpy())
    modified_var.append(df['sd'].pow(2).to_numpy())

# Stack the results of all the subjects together.
all_betas = df.index
modified_betas = np.stack(modified_betas)
modified_var = np.stack(modified_var)

# Display new contrasts of one subject
df.loc[new_params]

,mean,sd
conc_PCh+GPC_beta0,-0.000253,0.000804
conc_PCh+GPC_beta1,0.000100,0.001082
conc_PCh+GPC_beta2,0.002018,0.001441
conc_PCh+GPC_beta3,0.041277,0.000514
conc_Cr+PCr_beta0,-0.003022,0.001822
conc_Cr+PCr_beta1,0.003214,0.002359
conc_Cr+PCr_beta2,0.000033,0.003100
conc_Cr+PCr_beta3,0.352184,0.001237
conc_NAA+NAAG_beta0,-0.000991,0.001964
conc_NAA+NAAG_beta1,0.003240,0.002604


## 2. Use the FSL FLAMEO tool to carry out the higher-level statistical analysis

The `fsl_mrs.utils.fmrs_tools` package contains a wrapper function for FLAMEO. FLAMEO sits at the core of FSL `FEAT`'s higher level analysis routines.

But first we need to define our second-level design.

### 2.1 Second-level design and contrast matrices

Here we define a design matrix and contrasts to perform a paired t-test between all the betas of the STIM and CTRL conditions.

For information on the formation of these designs please see the [FSL course](https://open.win.ox.ac.uk/pages/fslcourse/website/online_materials.html) and specifically the [FMRI2 E2 video](https://www.youtube.com/watch?v=-nf9Hcthnm8).

In [6]:
# Design matrix
# Design matrix for a paired t-test with 10 subjects.
# The first column encodes the differences, which the others encode the average value of each subject
# across both conditions.
des_mat = np.zeros((20,11), int)
des_mat[:10, 0] = 1
des_mat[10:, 0] = -1
des_mat[:10, 1:] = np.eye(10)
des_mat[10:, 1:] = np.eye(10)
print(des_mat)

# Contrasts
# 1 = STIM>CTRL
# 2 = CTRL>STIM
# 3 = MEAN
# 4 = STIM
# 5 = CTRL
con_mat = np.zeros((5,11), float)
con_mat[0, 0] = 1
con_mat[1, 0] = -1
con_mat[2, 1:] = 0.1
con_mat[3, 0] = 1
con_mat[3, 1:] = 0.1
con_mat[4, 0] = -1
con_mat[4, 1:] = 0.1
print(con_mat)


[[ 1  1  0  0  0  0  0  0  0  0  0]
 [ 1  0  1  0  0  0  0  0  0  0  0]
 [ 1  0  0  1  0  0  0  0  0  0  0]
 [ 1  0  0  0  1  0  0  0  0  0  0]
 [ 1  0  0  0  0  1  0  0  0  0  0]
 [ 1  0  0  0  0  0  1  0  0  0  0]
 [ 1  0  0  0  0  0  0  1  0  0  0]
 [ 1  0  0  0  0  0  0  0  1  0  0]
 [ 1  0  0  0  0  0  0  0  0  1  0]
 [ 1  0  0  0  0  0  0  0  0  0  1]
 [-1  1  0  0  0  0  0  0  0  0  0]
 [-1  0  1  0  0  0  0  0  0  0  0]
 [-1  0  0  1  0  0  0  0  0  0  0]
 [-1  0  0  0  1  0  0  0  0  0  0]
 [-1  0  0  0  0  1  0  0  0  0  0]
 [-1  0  0  0  0  0  1  0  0  0  0]
 [-1  0  0  0  0  0  0  1  0  0  0]
 [-1  0  0  0  0  0  0  0  1  0  0]
 [-1  0  0  0  0  0  0  0  0  1  0]
 [-1  0  0  0  0  0  0  0  0  0  1]]
[[ 1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [-1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 1.   0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [-1.   0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1 

Note that we don't expect different variance between the paired data so we will use the default (all same group) covariance input.

### 2.2 Run FLAMEO

This will provide the group level contrasts (COPE), variances (VARCOPE), z-statistics and associated p-values, for each of the two contrasts.

Don't worry about the complexity of the output, we will refine it in the next steps!

In [7]:
import pandas as pd

p, z, out_cope, out_varcope = fmrs.flameo_wrapper(
        modified_betas,
        modified_var,
        design_mat=des_mat,
        contrast_mat=con_mat)

# Assemble the results into a nicely formatted pandas dataframe
all_stats = np.stack([out_cope, out_varcope, z, p])
columns = pd.MultiIndex.from_product(
        (['COPE', 'VARCOPE', 'z', 'p'],
        ['STIM>CTRL', 'CTRL>STIM', 'MEAN', 'STIM', 'CTRL']),
        names=['Statistics', 'Contrast'])
group_stats = pd.DataFrame(all_stats.reshape(-1, all_stats.shape[-1]), index=columns, columns=all_betas).T

# Filter just to see the mean activation stats
group_stats.filter(regex='mean_activation',axis=0)

Statistics                         COPE                                \
Contrast                      STIM>CTRL CTRL>STIM      MEAN      STIM   
conc_Asc_mean_activation       0.000367 -0.000367 -0.000032  0.000336   
conc_Asp_mean_activation      -0.004396  0.004396 -0.005627 -0.010023   
conc_Cr_mean_activation       -0.001253  0.001253 -0.001515 -0.002768   
conc_GABA_mean_activation      0.000580 -0.000580  0.000870  0.001450   
conc_GPC_mean_activation       0.000441 -0.000441  0.000912  0.001353   
conc_GSH_mean_activation      -0.000153  0.000153 -0.000351 -0.000504   
conc_Glc_mean_activation      -0.004822  0.004822 -0.004275 -0.009097   
conc_Gln_mean_activation       0.001393 -0.001393 -0.000420  0.000973   
conc_Glu_mean_activation       0.005488 -0.005488  0.007197  0.012685   
conc_Ins_mean_activation      -0.000302  0.000302 -0.000900 -0.001201   
conc_Lac_mean_activation       0.004809 -0.004809  0.004143  0.008952   
conc_Mac_mean_activation       0.000032 -0.000032 -0.000023  0.000009   
conc_NAA_mean_activation       0.000254 -0.000254  0.000450  0.000704   
conc_NAAG_mean_activation     -0.000563  0.000563 -0.000225 -0.000789   
conc_PCh_mean_activation      -0.000256  0.000256 -0.000920 -0.001176   
conc_PCr_mean_activation       0.001288 -0.001288  0.001453  0.002741   
conc_PE_mean_activation       -0.000839  0.000839 -0.000491 -0.001330   
conc_Scyllo_mean_activation    0.000291 -0.000291  0.000209  0.000500   
conc_Tau_mean_activation       0.000355 -0.000355 -0.000566 -0.000210   
conc_PCh+GPC_mean_activation   0.000184 -0.000184 -0.000008  0.000177   
conc_Cr+PCr_mean_activation    0.000039 -0.000039 -0.000071 -0.000032   
conc_NAA+NAAG_mean_activation -0.000304  0.000304  0.000223 -0.000081   
conc_Glu+Gln_mean_activation   0.006860 -0.006860  0.006772  0.013632   

Statistics                                    VARCOPE                \
Contrast                           CTRL     STIM>CTRL     CTRL>STIM   
conc_Asc_mean_activation      -0.000399  8.016947e-07  8.016947e-07   
conc_Asp_mean_activation      -0.001230  2.146597e-06  2.146597e-06   
conc_Cr_mean_activation       -0.000262  1.044820e-06  1.044820e-06   
conc_GABA_mean_activation      0.000290  2.187220e-06  2.187220e-06   
conc_GPC_mean_activation       0.000472  2.569311e-07  2.569311e-07   
conc_GSH_mean_activation      -0.000198  2.646742e-07  2.646742e-07   
conc_Glc_mean_activation       0.000547  9.385102e-07  9.385102e-07   
conc_Gln_mean_activation      -0.001813  1.599395e-06  1.599395e-06   
conc_Glu_mean_activation       0.001708  1.370661e-06  1.370661e-06   
conc_Ins_mean_activation      -0.000598  3.285828e-07  3.285828e-07   
conc_Lac_mean_activation      -0.000666  1.156286e-06  1.156286e-06   
conc_Mac_mean_activation      -0.000055  5.512133e-09  5.512133e-09   
conc_NAA_mean_activation       0.000195  1.933800e-07  1.933800e-07   
conc_NAAG_mean_activation      0.000338  1.483837e-07  1.483837e-07   
conc_PCh_mean_activation      -0.000664  2.559860e-07  2.559860e-07   
conc_PCr_mean_activation       0.000165  1.137120e-06  1.137120e-06   
conc_PE_mean_activation        0.000347  1.330151e-06  1.330151e-06   
conc_Scyllo_mean_activation   -0.000082  4.081656e-08  4.081656e-08   
conc_Tau_mean_activation      -0.000921  7.811558e-07  7.811558e-07   
conc_PCh+GPC_mean_activation  -0.000192  3.161982e-08  3.161982e-08   
conc_Cr+PCr_mean_activation   -0.000110  1.582826e-07  1.582826e-07   
conc_NAA+NAAG_mean_activation  0.000528  1.902349e-07  1.902349e-07   
conc_Glu+Gln_mean_activation  -0.000089  1.413004e-06  1.413004e-06   

Statistics                                                               \
Contrast                               MEAN          STIM          CTRL   
conc_Asc_mean_activation       7.992483e-07  1.565218e-06  1.636668e-06   
conc_Asp_mean_activation       2.140124e-06  4.194013e-06  4.379429e-06   
conc_Cr_mean_activation        1.043039e-06  2.005990e-06  2.169727e-06   
conc_GABA_mean_activati

## 3. Display results

### 3.1 Format the data for viewing
The above is still quite hard to look at. We can use further pandas tools to format the results more clearly, and also express changes as a percentage of the mean metabolite concentration

In [24]:
def format_df(df_in, metabs_to_plot=None):
    pd.set_option('mode.chained_assignment', None)

    # Remove the individual components of combined metabolites (except Glu & Gln)
    var_tmp = df_in\
    .loc[~(df_in.index.str.contains('_NAA_', case=False)\
        |df_in.index.str.contains('_NAAG_', case=False)\
        |df_in.index.str.contains('_Cr_', case=False)\
        |df_in.index.str.contains('_PCr_', case=False)\
        |df_in.index.str.contains('_PCh_', case=False)\
        |df_in.index.str.contains('_GPC_', case=False)),:]

    var_to_plot = var_tmp.filter(regex='_mean_activation',axis=0)
    var_to_plot.index = var_to_plot.index.str.replace('_mean_activation','')
    var_to_plot.index = var_to_plot.index.str.replace('conc_','')

    tmp_means = var_tmp.filter(regex='(?<!sigma_\d_)beta3',axis=0).COPE.abs().MEAN
    
    tmp_means.index = tmp_means.index.str.replace('conc_','')
    tmp_means.index = tmp_means.index.str.replace('_beta3','')
    # import pdb; pdb.set_trace()
    var_to_plot.COPE = var_to_plot.COPE.divide(tmp_means, axis=0).multiply(100)
    var_to_plot.VARCOPE = var_to_plot.VARCOPE.pow(0.5).divide(tmp_means, axis=0).multiply(100)

    # Drop the mean contrast
    var_to_plot = var_to_plot.drop(['MEAN', 'STIM', 'CTRL'], level=1, axis=1)
    
    var_to_plot = var_to_plot.sort_index()

    var_to_plot = var_to_plot.rename(columns={'COPE':'Effect(%)','VARCOPE':'SD(%)'})

    if metabs_to_plot:
        var_to_plot = var_to_plot.loc[metabs_to_plot]

    return var_to_plot.style\
    .format(formatter={('Effect(%)', 'STIM>CTRL'): "{:+06.2f}", ('Effect(%)', 'CTRL>STIM'): "{:+06.2f}",
                       ('SD(%)', 'STIM>CTRL'): "{:04.1f}", ('SD(%)', 'CTRL>STIM'): "{:04.1f}",
                       ('z', 'STIM>CTRL'):    "{:+05.2f}", ('z', 'CTRL>STIM'):    "{:+05.2f}",
                       ('p', 'STIM>CTRL'):    "{:05.3f}", ('p', 'CTRL>STIM'):    "{:05.3f}"})\
    .highlight_between(subset=['p'],left=0, right=0.05, props='font-weight:bold;color:#e83e8c')\
    .set_table_attributes("style='display:inline'")

The above `format_df` function expresses the COPE and VARCOPE as a % effect and % standard deviation on that value.
The significant p values (P<0.05) are highlighted.

In [26]:
format_df(group_stats)

## 3.2 Plotting

This section displays the group and individual subject time courses as fitted by the dynamic fitting.

There's a lot of plotting code here, but very little data manipulation takes place. Predominantly it selects out the data from the 
pandas Dataframe that we want to plot.

In [32]:
# Lots of utility code in this cell. Not important for understanding.
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 30})

# Assemble single-subject data into dataframe
mi = pd.MultiIndex.from_product([['STIM', 'CTRL'], [f'sub{x}' for x in range(10)]], names=['Condition', 'Subject#'])
single_sub_betas = pd.DataFrame(modified_betas, columns=all_betas, index=mi).T

# Load in original first-level design matrix
design_matrix = pd.read_csv('designmat.csv',header=None).to_numpy()


def plot_2ndlvl(metab, df, percent=True):
    """Returns the temporal response of a metabolite or other free parameter

    :param metab: Regex search string for filtering GLM fitted metabolite/parameter
    :type metab: str
    :param df: stats dataframe
    :type df: pd.DataFrame
    :param percent: Scale to constant beta in %, defaults to True
    :type percent: bool, optional
    """
    metab = metab + 'beta'
    currcopes = df.filter(regex=metab, axis=0).COPE
    curr95ci = df.filter(regex=metab, axis=0).VARCOPE.pow(0.5)
    curr95ci_low = currcopes - curr95ci
    curr95ci_high = currcopes + curr95ci

    constant_term = f'_beta{design_matrix.shape[1]-1}'

    if percent:      
        curr95ci_low = (curr95ci_low / currcopes.filter(regex=constant_term, axis=0).to_numpy())
        curr95ci_low = curr95ci_low.to_numpy()
        
        curr95ci_high = (curr95ci_high / currcopes.filter(regex=constant_term, axis=0).to_numpy())
        curr95ci_high = curr95ci_high.to_numpy()
        
        currcopes = (currcopes / currcopes.filter(regex=constant_term, axis=0).to_numpy())
        currcopes = currcopes.to_numpy()
        return 100 * (np.dot(design_matrix,currcopes)  - 1.0)\
             , 100 * (np.dot(design_matrix,curr95ci_low)  - 1.0)\
             , 100 * (np.dot(design_matrix,curr95ci_high)  - 1.0)
    else:
        currcopes = currcopes.to_numpy()
        curr95ci_low = curr95ci_low.to_numpy()
        curr95ci_high = curr95ci_high.to_numpy()
        return np.dot(design_matrix,currcopes), np.dot(design_matrix,curr95ci_low), np.dot(design_matrix,curr95ci_high)

def indiv_traces(metab, df, percent=True):
    vals = []
    for subj in df.index.unique(level=0):
        p = df.loc[subj].filter(regex=metab+'beta', axis=0).values
        # import pdb; pdb.set_trace()
        val = np.dot(p, design_matrix.T)
        if percent:
            val = 100 * val/p[-1] - 100
        vals.append(val)
    return vals

# ## Plot 
# colors = plt.get_cmap('tab10').colors
# t_axis = np.arange(0,64)

# # Plot glutamate response
# met = 'Glu'
# stim_df = group_stats.xs('STIM',axis=1, level='Contrast')
# plt.plot(plot_2ndlvl(f'_{met}_', stim_df)[0], color=colors[1], label=met, linewidth=5)
# plt.fill_between(
#     t_axis,
#     plot_2ndlvl(f'_{met}_', stim_df)[1],
#     plot_2ndlvl(f'_{met}_', stim_df)[2],
#     color=colors[1],
#     alpha=0.1)

# for it in indiv_traces(f'_{met}_', single_sub_betas.STIM.T, percent=True):
#     plt.plot(it, linestyle=':', color=colors[1], linewidth=0.5)

# Plotly (interactive plotting) utility function
from plotly.subplots import make_subplots
import plotly.graph_objects as go

colors = plt.get_cmap('tab10').colors
def color_t2s(tin, alpha=1):
    tscaled = [255*t for t in tin]
    return f'rgba({tscaled[0]}, {tscaled[1]}, {tscaled[2]}, {alpha})'
line_size = dict(data=3, indivdata=0.5)

def plotly_glm_plot(main_df, indiv_df, stim_or_ctrl, metab_list):

    cur_df = main_df.xs(stim_or_ctrl,axis=1, level='Contrast')

    traces = []
    axis = np.arange(1, 65)
    for idx, met in enumerate(metab_list):
        regexstr = f'_{met}_'
        regexstr = regexstr.replace('+', '\+')

        data = plot_2ndlvl(regexstr, cur_df)
        mean_trace = go.Scatter(x=axis, y=data[0],
                    mode='lines',
                    name=met,
                    legendgroup=met,
                    line=dict(
                        color=color_t2s(colors[idx]),
                        width=line_size['data']),
                    )
        traces.append(mean_trace)
        errors = go.Scatter(
            x=np.concatenate((axis, axis[::-1])) , # x, then x reversed
            y=np.concatenate((data[2], data[1][::-1])), # upper, then lower reversed
            fill='toself',
            fillcolor=color_t2s(colors[idx], 0.2),
            line=dict(color='rgba(255,255,255,0)'),
            hoverinfo="skip",
            name=met + ' CI',
            legendgroup=met,
            showlegend=False
        )
        traces.append(errors)
        for jdx, it in enumerate(indiv_traces(regexstr, indiv_df[stim_or_ctrl].T, percent=True)):
            ind_trace = go.Scatter(
                x=axis, y=it,
                mode='lines',
                name=met + f'_S{jdx}',
                legendgroup=met,
                line=dict(
                    color=color_t2s(colors[idx],0.5),
                    width=line_size['indivdata']),
                showlegend=False)
            traces.append(ind_trace)

    fig = make_subplots(rows=2, cols=1,
                    row_heights=[0.6, 0.4],
                    vertical_spacing=0.1,
                    specs=[[{'type': 'scatter'}], [{'type': 'scatter'}]])

    for trace in traces:
        fig.add_trace(trace, row=1, col=1)

    lw_traces = []
    data = plot_2ndlvl('sigma_0_', cur_df, percent=False)
    lw_traces.append(go.Scatter(x=axis, y=data[0]/np.pi,
                mode='lines',
                name='lw: sigma',
                legendgroup='sigma',
                line=dict(
                    color=color_t2s((0,0,0)),
                    width=line_size['data']),
                ))
    lw_traces.append(go.Scatter(
            x=np.concatenate((axis, axis[::-1])) , # x, then x reversed
            y=np.concatenate((data[2], data[1][::-1]))/np.pi, # upper, then lower reversed
            fill='toself',
            fillcolor=color_t2s((0,0,0), 0.2),
            line=dict(color='rgba(255,255,255,0)'),
            hoverinfo="skip",
            name='lw: sigma CI',
            legendgroup='sigma',
            showlegend=False
        ))
    for jdx, it in enumerate(indiv_traces('sigma_0_', indiv_df[stim_or_ctrl].T, percent=False)):
        lw_traces.append(go.Scatter(
            x=axis, y=it/np.pi,
            mode='lines',
            name=f'lw: sigma_S{jdx}',
            legendgroup='sigma',
            line=dict(
                color=color_t2s((0,0,0),0.5),
                width=line_size['indivdata']),
            showlegend=False))

    for trace in lw_traces:
        fig.add_trace(trace, row=2, col=1)


    fig.update_layout(template='plotly_white')

    fig.update_xaxes(
        row=1,
        tick0=0, dtick=16,
        range=[0, 65])
    fig.update_xaxes(
        row=2,
        title_text='Transient #',
        tick0=0, dtick=16,
        range=[0, 65])

    fig.update_yaxes(row=1,
                    title_text='% change',
                    zeroline=True,
                    zerolinewidth=0.5,
                    zerolinecolor='Gray',
                    range=[-40, 40])

    fig.update_yaxes(row=2,
                    title_text='Gaussian Linewidth (Hz)',
                    range=[3.5, 5.5])


    fig.layout.update({'height': 700})
    fig.update_layout(
        margin=dict(l=30, r=30, t=30, b=30),
    )
    return fig

In [35]:
# Plotting control
stim_or_ctrl = 'STIM'  # 'STIM' or 'CTRL'
metab_list = ['NAA+NAAG', 'Glu', 'Asp', 'Lac', 'Glc']

plotly_glm_plot(group_stats, single_sub_betas, stim_or_ctrl, metab_list)


# Part 2: Using the command line interface

This final section takes you through the steps needed to replicate the above analysis steps using the FSL-MRS command line scripts, specifically `fmrs_stats`.

## A. Create auxiliary files
We will create four files as inputs to the main `fmrs_stats` script. These are:
1. A list of directories containing the results files.
2. The first-level contrasts, described in a JSON formatted file.
3. The higher-level design matrix, in the FSL VEST format.
4. The higher-level contrast matrix, again in the FSL VEST format.

We start with #1, the list of dynamic fit results.

In [1]:
%%writefile second_level_files/results_list
first_level_results/sub0/stim
first_level_results/sub1/stim
first_level_results/sub2/stim
first_level_results/sub3/stim
first_level_results/sub4/stim
first_level_results/sub5/stim
first_level_results/sub6/stim
first_level_results/sub7/stim
first_level_results/sub8/stim
first_level_results/sub9/stim
first_level_results/sub0/ctrl
first_level_results/sub1/ctrl
first_level_results/sub2/ctrl
first_level_results/sub3/ctrl
first_level_results/sub4/ctrl
first_level_results/sub5/ctrl
first_level_results/sub6/ctrl
first_level_results/sub7/ctrl
first_level_results/sub8/ctrl
first_level_results/sub9/ctrl

Writing second_level_files/results_list


The next cell creates #2, the first-level contrasts. This is formatted like the interactive python Dataclass, but encoded in the JSON format. Multiple contrasts can be listed. Give the contrast a descriptive name!

In [2]:
%%writefile second_level_files/fl_contrasts.json
[
        {
            "name": "mean_activation",
            "betas": ["beta0", "beta1"],
            "scale": [0.5, 0.5]
        }
]

Writing second_level_files/fl_contrasts.json


The last files are #3 and #4, the higher-level designs. These are constructed as above (though here we just include the STIM>CTRL and CTRL>STIM contrasts).

Once written as a text file, they must be converted tot he VEST format using the `Text2Vest` script packaged with FSL.

In [4]:
import numpy as np
des_mat = np.zeros((20,11), int)
des_mat[:10, 0] = 1
des_mat[10:, 0] = -1
des_mat[:10, 1:] = np.eye(10)
des_mat[10:, 1:] = np.eye(10)
print(des_mat)
np.savetxt('pttest_design', des_mat)

%sx Text2Vest pttest_design second_level_files/design.mat

con_mat = np.zeros((2,11), int)
con_mat[0, 0] = 1
con_mat[1, 0] = -1
print(con_mat)
np.savetxt('pttest_contrasts', con_mat)

%sx Text2Vest pttest_contrasts second_level_files/design.con

[[ 1  1  0  0  0  0  0  0  0  0  0]
 [ 1  0  1  0  0  0  0  0  0  0  0]
 [ 1  0  0  1  0  0  0  0  0  0  0]
 [ 1  0  0  0  1  0  0  0  0  0  0]
 [ 1  0  0  0  0  1  0  0  0  0  0]
 [ 1  0  0  0  0  0  1  0  0  0  0]
 [ 1  0  0  0  0  0  0  1  0  0  0]
 [ 1  0  0  0  0  0  0  0  1  0  0]
 [ 1  0  0  0  0  0  0  0  0  1  0]
 [ 1  0  0  0  0  0  0  0  0  0  1]
 [-1  1  0  0  0  0  0  0  0  0  0]
 [-1  0  1  0  0  0  0  0  0  0  0]
 [-1  0  0  1  0  0  0  0  0  0  0]
 [-1  0  0  0  1  0  0  0  0  0  0]
 [-1  0  0  0  0  1  0  0  0  0  0]
 [-1  0  0  0  0  0  1  0  0  0  0]
 [-1  0  0  0  0  0  0  1  0  0  0]
 [-1  0  0  0  0  0  0  0  1  0  0]
 [-1  0  0  0  0  0  0  0  0  1  0]
 [-1  0  0  0  0  0  0  0  0  0  1]]
[[ 1  0  0  0  0  0  0  0  0  0  0]
 [-1  0  0  0  0  0  0  0  0  0  0]]


[]

## B. Run the command line script
We pass the files created above to the `fmrs_stats` script, as well as defining and output location `--output` and any metabolites to combine `--combine`.

In [5]:
%%sx
fmrs_stats\
    --data second_level_files/results_list\
    --output group_results\
    --fl-contrasts second_level_files/fl_contrasts.json\
    --combine NAA NAAG\
    --combine Cr PCr\
    --combine PCh GPC\
    --combine Glu Gln\
    --hl-design second_level_files/design.mat\
    --hl-contrasts second_level_files/design.con\
    --hl-contrast-names "STIM>CTRL" "CTRL>STIM"\
    --overwrite

[]

## C. Load the results
There are also folders for each subject/run containing the results of the first-level contrast and metabolite combinations.

In [7]:
import pandas as pd
# Load the results
cmd_line_df = pd.read_csv('group_results/group_stats.csv',index_col=0, header=[0,1])

# Format for display
cmd_line_df.filter(regex='mean_activation', axis=0)\
.style\
.format(formatter={('z', 'STIM>CTRL'):    "{:+05.2f}", ('z', 'CTRL>STIM'):    "{:+05.2f}",
                   ('p', 'STIM>CTRL'):    "{:05.3f}", ('p', 'CTRL>STIM'):    "{:05.3f}"})\
.highlight_between(subset=['p'],left=0, right=0.05, props='font-weight:bold;color:#e83e8c')